In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);


In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like TGLF saturation rules
sty,act = FUSE.study_parameters(:TGLFdb);
sty

In [ ]:
# Interacting with sty
sty.server = "saga"
sty.n_workers = 1

sty.database_folder = "/mnt/beegfs/users/neisert/ODSs/iri/166066" #166066" #mastu_test" #d3d_negDcake_oak"#"/mnt/beegfs/users/neisert/ODSs/mastu"

sty.save_folder = "/mnt/beegfs/users/neisert/ODSs/iri/166066/outputs_gknn_rho_0p05_0p02_0p85_rot" #"/mnt/beegfs/users/neisert/ODSs/mastu/outputs_sat1_es_d3d"
sty.sat_rules = missing #[:sat2]#,:sat1,:sat2,:sat3] #study specific parameters
# It's also possible to run with a custom tglfnn model, set sty.custom_tglf_models
sty.custom_tglf_models = ["sat2_em_d3d_azf+1","sat3_em_d3d_azf+1","sat2_em_d3d_azf-1","sat3_em_d3d_azf-1"]
sty.file_save_mode = :overwrite
sty.release_workers_after_run = true # this is the default behavior and releases workers after running the study
sty

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyTGLFdb(sty, act; n_workers=1); # it is possible to pass in keyword arguments to sty

# instantiating the study will also setup the study i.e. FUSE.setup(study);

In [ ]:
# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE


In [ ]:
# The filename you provided
#=filename = "/mnt/beegfs/users/neisert/ODSs/d3d_negDcake_oak/fuse_prepared_inputs/ods_193899_2810.json"

timefn = match(r"(\d+)\.\w+$", filename)
if timefn !== nothing
    timef = timefn.captures[1]
else
    timef = 0
end
println("$timef")
# Alternative regular expression to capture numbers
alternative_match = match(r"(\d+)\.json$", filename)

if alternative_match !== nothing
    # Extract the number
    alternative_number = parse(Int, alternative_match.captures[1])
    println("Alternative match found: ", alternative_number)
else
    println("Alternative match not found.")
end
=#

In [ ]:
study.act.ActorFluxMatcher.evolve_rotation = :flux_match
study.act.ActorFluxMatcher.rho_transport = 0.05:0.02:0.85
# density is evolved now by default with impurities fixed
#=study.act.ActorFluxMatcher.evolve_densities = Dict(
    :D        => :quasi_neutrality,
    :D_fast   => :fixed,
    :C        => :fixed,
    :electrons => :flux_match)
=#
study.act.ActorFluxMatcher

In [ ]:
FUSE.run(study);   # runs the study, this also writes the dataframes

# at the end of the run workers are released so you will have to setup again if you want to run more

In [ ]:
# data for this workflow is stored in a dictionary of dataframes study.dataframes_dict
#study.dataframes_dict[:sat1]
#study.dataframes_dict["sat1_es_d3d"]

study.dataframes_dict["sat2_em_d3d_azf+1","sat3_em_d3d_azf+1","sat2_em_d3d_azf-1","sat3_em_d3d_azf-1"]

In [ ]:
# analyze the study results 
FUSE.analyze(study);